### 0. Environment Setup and Model Initialization

In [ ]:
import os
import re
import pandas as pd
import torch
import subprocess

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score
from sentence_transformers import SentenceTransformer, util

In [1]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home5/b09303131/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Device: cuda
NVIDIA RTX A6000


In [ ]:
with open("password.txt") as f:
    hf_token = f.read()
subprocess.run(["huggingface-cli", "login", "--token", hf_token], check=True)

In [ ]:
with open('thesis_output.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    
with open("apikey.txt", 'r', encoding='utf-8') as file:
    key = file.read()
os.environ["OPENAI_API_KEY"] = key

In [9]:
chunked_docs = content.split("###")
openai_llm = ChatOpenAI(model="gpt-4o")
embeddings_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
db = FAISS.from_texts(chunked_docs, embeddings_model)

### 1. EconLitQA Functions and Examples Display

In [10]:
def keyword_extraction(input):
    # 1. Keyword Extraction
    keyword_extraction_prompt = PromptTemplate(
        input_variables=["query"],
        template="""
        假設你是一位專業助手，請仔細閱讀以下問題。如果有人要根據這些問題來檢索經濟學論文，請給出三個以下可以幫助找到相關論文的核心關鍵字，並按照編號列出，請不要提供題目沒有提到的關鍵字。

        舉例來說：
        問題：
        假設你是一位經濟學領域的研究者，請根據提供的的資料，分析行為經濟學中決策理論的應用，並根據提供的資料提出可能的研究方向。**請用繁體中文回答**

        關鍵字：
        1. 行為經濟學
        2. 決策理論

        問題：
        假設你是一位經濟學領域的研究者，請根據提供的的資料，觀察有關最低工資政策對就業影響的研究，並找出相關的研究熱點。**請用繁體中文回答**

        關鍵字：
        1. 最低工資
        2. 就業影響

        問題：
        假設你是一位經濟學領域的研究者，請根據提供的的資料，分析環境經濟學中碳排放交易市場的研究現況，並找出相關文章的研究熱點。**請用繁體中文回答**

        關鍵字：
        1. 環境經濟學
        2. 碳排放交易

        問題：
        假設你是一位經濟學領域的研究者，請根據提供的的資料，分析貿易保護政策對全球供應鏈的影響，並根據資料找出核心研究方向。**請用繁體中文回答**

        關鍵字：
        1. 貿易保護
        2. 全球供應鏈

        現在換你回答：
        問題：
        {query}

        關鍵字：
        """
        )   


    keyword_chain = keyword_extraction_prompt | openai_llm

    query = "假設你是一位經濟學領域的學者，請根據提供的的資料，"
    query += input
    query += '**請用繁體中文回答**'
    keywords = keyword_chain.invoke({"query": query})

    keywords_list = re.findall(r"\d+\.\s*(.+)", keywords.content)
    cleaned_keywords = []
    for keyword in keywords_list:
        cleaned_keyword = re.sub(r"[^\u4e00-\u9fa5]", "", keyword) 
        cleaned_keywords.append(cleaned_keyword)

    formatted_keywords = " ".join(cleaned_keywords[:3])
    return formatted_keywords, query

def retrieve(formatted_keywords):
    # 2. Retrieve Documents Chunks
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7, "k": 20})
    retrieved_docs = retriever.invoke(formatted_keywords)
    retrieved_content = "\n".join([doc.page_content for doc in retrieved_docs])
    return retrieved_content

def final_qa(keywords, retrieved_content, query):
    # 3. Final QA
    final_prompt_template = PromptTemplate(
        input_variables=["keywords", "retrieved_content", "query"],
        template="""
        假設你是一位經濟學領域的學者。以下是根據檢索關鍵字 "{keywords}" 從資料庫中獲取的相關內容，若有跟問題無關之資料，請忽略它：

        {retrieved_content}

        請根據以上內容回答以下問題：
        {query}
        若文件中無法找到相關資訊，則請務必回覆「資訊不足」。另外，也不要生成不存在的文獻，僅提供實際存在的學術研究。
        **請務必使用繁體中文回答。**
        """
    )

    final_qa_chain = final_prompt_template | openai_llm
    response = final_qa_chain.invoke({"keywords": keywords, "retrieved_content": retrieved_content, "query": query})
    return response.content

In [11]:
# Display 5 Examples
inputs = ["請幫我找到入學管道這個主題相關的論文中找出最常出現的學術期刊或出版單位，並分析這些來源的影響力。",
         "請幫我找到入學管道這個主題最常使用的關鍵字，並推測這些關鍵字可能的研究熱點。",
         "請幫我以入學管道這個主題的相關文章綜合關鍵字、摘要和來源，是否能觀察到這個主題的研究熱點或趨勢。",
         "幫我觀察哪些作者在入學管道這個主題發表的文章最多",
         "幫我寫一篇有關入學管道的文獻回顧。"]

keywords = []
querys = []
retrieved_contents = []
responses =[]
for i in range(5):
    keyword, query = keyword_extraction(inputs[i])
    print(f"{i + 1}.")
    print("query:", query)
    print("keywords", keyword)
    retrieved_content = retrieve(keyword)
    response = final_qa(keyword, retrieved_content, query)
    print("Final Answer:")
    print(response)   
    keywords.append(keyword)
    querys.append(query)
    retrieved_contents.append(retrieved_content)
    responses.append(response)

1.
query: 假設你是一位經濟學領域的學者，請根據提供的的資料，請幫我找到入學管道這個主題相關的論文中找出最常出現的學術期刊或出版單位，並分析這些來源的影響力。**請用繁體中文回答**
keywords 入學管道 學術期刊 出版單位
Final Answer:
根據提供的資料，關於「入學管道」這個主題，最常出現的學術期刊是《經濟論文叢刊》。在資料中，多篇文章來自於此期刊，包括：

1. 《入學管道與學習表現》
2. 《大學入學管道與學業表現－以北部某私立大學為例》
3. 《誰是台大學生？（2001-2014）－多元入學的影響》
4. 《高中教育生產函數的估計—學校品質與家庭資源的投入產出分析》
5. 《2001年高中職和五專多元入學方案與入學後學校滿意度、學業壓力及在校表現之關係》

《經濟論文叢刊》作為一個學術期刊，顯示出其在經濟學領域內的影響力，尤其是在研究台灣教育制度與入學管道相關議題方面，具有相當的代表性和學術價值。該期刊刊載的文章涉及多方面的分析，包括學業表現、家庭背景、學校滿意度等，這表明該期刊對於分析教育經濟學議題的廣度與深度。

因此，《經濟論文叢刊》在這一主題的研究中扮演了重要角色，提供了豐富的實證研究和理論分析，對於學術界和政策制定者均具有重要的參考價值。
2.
query: 假設你是一位經濟學領域的學者，請根據提供的的資料，請幫我找到入學管道這個主題最常使用的關鍵字，並推測這些關鍵字可能的研究熱點。**請用繁體中文回答**
keywords 入學管道 教育政策 高等教育
Final Answer:
根據提供的資料，有關入學管道主題最常使用的關鍵字包括「多元入學」、「個人申請」、「繁星推薦」、「指考入學」、「學業表現」和「家庭經濟背景」。這些關鍵字暗示了一些可能的研究熱點：

1. **多元入學制度的影響**：許多研究關注多元入學政策對學生學業表現、教育機會和社會公平的影響。例如，研究探討不同入學管道（如個人申請、繁星推薦、指考入學）對學業表現的差異以及對教育公平性的影響。

2. **家庭背景與入學管道的關聯**：有些文獻分析家庭經濟背景、社經地位對選擇不同入學管道的影響，這涉及教育資源分配和社會流動的議題。

3. **入學管道與學生特性**：研究也探討不同入學管道的學生特性，分析哪些學生傾向選擇特定管道，並進一步探討這些選擇背後的因素

### 2. Faithfulness Evaluation for some Examples

In [12]:
def decompose_to_statements(question, answer):
    decompose_prompt_template = PromptTemplate(
        input_variables=["question", "answer"],
        template="""
        根據以下的問題與回答，請從回答中創建一個或多個具體的陳述（每句話拆分為具體簡短的陳述句），並按照編號列出。
        問題：{question}
        回答：{answer}
        """
    )
    decompose_chain = decompose_prompt_template | openai_llm
    statement = decompose_chain.invoke({"question": question, "answer": answer})
    return statement

def verify_statements(context, statements):
    verify_prompt_template = PromptTemplate(
        input_variables=["statement", "context"],
        template="""
        考慮以下的上下文與陳述句，判斷每個陳述句是否被上下文中的資訊支持。
        請為每個陳述句判斷正確與否（請只要回答是或否，不要其他解釋），並按照編號列出。
        {statement}
        上下文：{context}
        """
    )
    verify_chain = verify_prompt_template | openai_llm
    bools = verify_chain.invoke({"statement": statements, "context":context})
    return bools

def calculate_faithfulness_score(context, question, answer):
    statements = decompose_to_statements(question, answer)
    verification = verify_statements(context, statements)  
    true = 0
    false = 0
    for line in verification.content.split("\n"):
        if "是" in line:
            true += 1
        if "否" in line:
            false += 1
    if true + false == 0:
        faithfulness_score = 0
    else:
        faithfulness_score = true / (true + false)
        faithfulness_score = round(faithfulness_score, 3)
    return faithfulness_score 

In [15]:
# Calculate faithfulness scores for each Examples
faithfulness_scores = []
for i in range(5):
    score = calculate_faithfulness_score(retrieved_contents[i], querys[i], responses[i])
    faithfulness_scores.append(score)

for i, score in enumerate(faithfulness_scores, start=1):
    print(f"問題 {i} 的信實度分數：{score}")

問題 1 的信實度分數：0.5
問題 2 的信實度分數：0.107
問題 3 的信實度分數：0.909
問題 4 的信實度分數：0.8
問題 5 的信實度分數：0.708


### 3. Answer Relevance Evaluation for some Examples

In [15]:
def question_generation(answer):
    question_prompt_template = PromptTemplate(
        input_variables=["answer"],
        template="""
        請根據以下答案產生一個對應的問題
        答案：{answer}
        """
    )
    verify_chain = question_prompt_template | openai_llm
    question = verify_chain.invoke({"answer":answer})
    return question

def calculate_answer_relevance_score(questions, pred_questions):
    embedding1 = embeddings_model.encode(questions, convert_to_tensor=True)
    embedding2 = embeddings_model.encode(pred_questions, convert_to_tensor=True)
    relevance_score = util.cos_sim(embedding1, embedding2).item()   
    return relevance_score

In [17]:
# Calculate answer relevance scores for each Examples
answer_relevance_scores = []
for i in range(5):
    pred_questions = question_generation(responses[i])
    score = calculate_answer_relevance_score(querys[i], pred_questions.content)
    answer_relevance_scores.append(score)

for i, score in enumerate(answer_relevance_scores, start=1):
    print(f"問題 {i} 的answer_relevance分數：{round(score,3)}")

問題 1 的answer_relevance分數：0.944
問題 2 的answer_relevance分數：0.947
問題 3 的answer_relevance分數：0.912
問題 4 的answer_relevance分數：0.912
問題 5 的answer_relevance分數：0.88


### 4. Overall Evaluation

##### 4.1 Keywords Evaluation

In [18]:
data = pd.read_csv("econ_questions_keywords.csv")
questions = data["questions"]
print(data.shape)
data.head(5)

(100, 2)


,questions,keywords
0,從社會保障政策的角度，分析其對貧富差距的潛在影響。,社會保障政策 貧富差距
1,請從金融市場波動性的角度，分析其對發展中國家經濟的深遠影響。,金融市場波動性 發展中國家經濟
2,請探討全球化影響與教育機會的關聯，並分析文獻中的爭議點。,全球化影響 教育機會
3,從歷史與當前數據的角度，分析技術進步對城市居民的影響。,技術進步 城市居民
4,從歷史與當前數據的角度，分析數位化對年輕一代的影響。,數位化 年輕一代


In [15]:
predicted_keywords = []
for i in range(len(data)):
    llm_generated_keywords, _ = keyword_extraction(data["questions"][i])  
    predicted_keywords.append(llm_generated_keywords)
    if (i + 1) % 5 == 0:
        print(f"{i + 1} questions are done!")

true_keywords = data["keywords"]

5 questions are done!
10 questions are done!
15 questions are done!
20 questions are done!
25 questions are done!
30 questions are done!
35 questions are done!
40 questions are done!
45 questions are done!
50 questions are done!
55 questions are done!
60 questions are done!
65 questions are done!
70 questions are done!
75 questions are done!
80 questions are done!
85 questions are done!
90 questions are done!
95 questions are done!
100 questions are done!


In [16]:
adjusted_predicted_keywords = []
for i in range(len(predicted_keywords)):
    adjusted_list = []
    true_kw_list = true_keywords[i].split()
    pred_kw_list = predicted_keywords[i].split()
    true_embeddings = embeddings_model.encode(true_kw_list, convert_to_tensor=True)
    pred_embeddings = embeddings_model.encode(pred_kw_list, convert_to_tensor=True)
    for j, pred_kw in enumerate(pred_kw_list):
        cos_similarities = util.cos_sim(pred_embeddings[j], true_embeddings).cpu().tolist()
        max_similarity = max(cos_similarities[0])
        if max_similarity > 0.9:
            max_idx = cos_similarities[0].index(max_similarity)
            adjust_kw = true_kw_list[max_idx]
            adjusted_list.append(adjust_kw)
        else:
            adjusted_list.append(pred_kw)
    adjusted_predicted_keywords.append(adjusted_list if adjusted_list else [])

In [17]:
# Convert string format to list format if needed
true_kw_lists = [kw.split() if isinstance(kw, str) else kw for kw in true_keywords]
pred_kw_lists = [kw.split() if isinstance(kw, str) else kw for kw in adjusted_predicted_keywords]

# Get unique keywords from both sets
all_keywords = set()
for kw_list in true_kw_lists + pred_kw_lists:
    all_keywords.update(kw_list)

# Create binary vectors
mlb = MultiLabelBinarizer(classes=list(all_keywords))
y_true = mlb.fit_transform(true_kw_lists)
y_pred = mlb.transform(pred_kw_lists)

# Calculate metrics
metrics = {
    'precision': precision_score(y_true, y_pred, average='micro'),
    'recall': recall_score(y_true, y_pred, average='micro'),
    'f1': f1_score(y_true, y_pred, average='micro'),
}


# Create a formatted results dataframe
results_df = pd.DataFrame({
    'Metric': list(metrics.keys()),
    'Score': list(metrics.values())
})

results_df

,Metric,Score
0,precision,0.879668
1,recall,0.986047
2,f1,0.929825


##### 4.2 Faithfulness Evaluation

In [20]:
keywords = []
querys = []
retrieved_contents = []
responses =[]
for i in range(len(questions)):
    keyword, query = keyword_extraction(questions[i])
    print(f"{i + 1}.")
    print("query:", query)
    print("keywords", keyword)
    retrieved_content = retrieve(keyword)
    response = final_qa(keyword, retrieved_content, query)
    print("Final Answer:")
    print(response)   
    keywords.append(keyword)
    querys.append(query)
    retrieved_contents.append(retrieved_content)
    responses.append(response)

1.
query: 假設你是一位經濟學領域的學者，請根據提供的的資料，從社會保障政策的角度，分析其對貧富差距的潛在影響。**請用繁體中文回答**
keywords 社會保障政策 貧富差距
Final Answer:
從提供的資料中，我們可以看到關於社會保障政策對貧富差距影響的幾個關鍵點。以下是一些分析：

1. **國民年金政策與中低收入家庭**：根據蔡彣涓的研究，國民年金政策對中低收入老人家庭的影響顯著，其總消費支出因而下降9%。這顯示了國民年金政策在某種程度上可能減輕了家庭的經濟負擔，但也可能反映了資源分配的不均，特別是不同性別的所得收入者在家庭資源分配上的差異性。這樣的政策對減少貧富差距的效果可能有限，需進一步考量如何提升中低收入家庭的實際可支配收入。

2. **年金改革與世代間不均**：董安琪和謝餘慶的研究指出，若不進行年金改革，世代間會出現極度不公的情況，並且國家財政將面臨崩壞。這表明現行的年金制度可能加劇了不同世代間的財富不平等，且改革方案若不具長遠目標，可能無法有效改善貧富差距。

3. **勞工保險老年給付的分配效果**：根據羅紀琼的分析，勞工保險的退休制度目前會使分配惡化，因為所得愈高的人獲得的退休金福祉愈大，這與社會保險的公平原則相悖。這意味著現行的社會保障政策在某種程度上可能擴大貧富差距，尤其是在退休後的分配上。

綜合上述資料，社會保障政策在設計和實施上需進一步考量其對不同收入階層的影響，尤其是如何有效減少貧富差距。政策的設計應該更具針對性，以確保低收入群體能夠真正受益，並減少制度對高收入群體的偏向。這樣才能夠促進社會的公平與和諧。
2.
query: 假設你是一位經濟學領域的學者，請根據提供的的資料，請從金融市場波動性的角度，分析其對發展中國家經濟的深遠影響。**請用繁體中文回答**
keywords 金融市場波動性 發展中國家 經濟影響
Final Answer:
根據所提供的資料，涉及發展中國家金融市場波動性對經濟影響的具體分析並不多。然而，幾篇文章提供了一些可以推斷相關影響的背景資訊：

1. **金融政策在雙元性經濟的效果**：這篇文章提到，開發中國家通常存在組織銀行體系與非正式借貸並存的雙元性金融體系。金融市場波動可能會影響這些國家的資金供應，進而影響經濟增長。當市場波動加劇時，可能導致資金成本上升，影響企業投資和消費者支出。


In [20]:
faithfulness_scores = []
for i in range(len(querys)):    
    score = calculate_faithfulness_score(retrieved_contents[i], querys[i], responses[i])
    faithfulness_scores.append(score)
    if (i + 1) % 5 == 0:
        print(f"{i + 1} questions are done!")

5 questions are done!
10 questions are done!
15 questions are done!
20 questions are done!
25 questions are done!
30 questions are done!
35 questions are done!
40 questions are done!
45 questions are done!
50 questions are done!
55 questions are done!
60 questions are done!
65 questions are done!
70 questions are done!
75 questions are done!
80 questions are done!
85 questions are done!
90 questions are done!
95 questions are done!
100 questions are done!


In [22]:
print(sum(faithfulness_scores)/len(faithfulness_scores))

0.8785500000000002


##### 4.3 Answer Relevance Evaluation

In [23]:
answer_relevance_scores = []
for i in range(len(responses)):
    pred_questions = question_generation(responses[i])
    score = calculate_answer_relevance_score(querys[i], pred_questions.content)
    answer_relevance_scores.append(score)
    if (i + 1) % 5 == 0:
        print(f"{i + 1} questions are done!")

5 questions are done!
10 questions are done!
15 questions are done!
20 questions are done!
25 questions are done!
30 questions are done!
35 questions are done!
40 questions are done!
45 questions are done!
50 questions are done!
55 questions are done!
60 questions are done!
65 questions are done!
70 questions are done!
75 questions are done!
80 questions are done!
85 questions are done!
90 questions are done!
95 questions are done!
100 questions are done!


In [32]:
average_relevance_scores = round(sum(answer_relevance_scores)/len(answer_relevance_scores), 3)
print(average_relevance_scores)

0.927
